In [11]:
import numpy as np
import time
import os
import random
import scipy.io as sio


In [12]:
#read vectorization data and Cd data
# Sd_array_total = np.genfromtxt('v_rep_140_density50.csv', delimiter=',')
# Cds_total = np.genfromtxt('Cds_140.csv', delimiter=',')

# fmat = sio.loadmat('vecs_with_cds.mat')
# print(fmat.keys())

In [13]:
Sd_array_total = np.genfromtxt('v_rep_140_density145.csv', delimiter=',')
Cds_total = np.genfromtxt('Cds_140.csv', delimiter=',')

In [14]:
number_sample = 140

In [15]:
# small = np.genfromtxt('small.csv', delimiter=',').astype(int)
# small[0] = 0
# big = np.genfromtxt('big.csv', delimiter=',').astype(int)
# big[0] = 1
# classic = np.genfromtxt('classic.csv', delimiter=',').astype(int)
# classic[0] = 5
# sports = np.genfromtxt('sports.csv', delimiter=',').astype(int)
# sports[0] = 6
# # print(small, big, classic, sports)


In [16]:
seed = np.random.randint(10000)
print(seed)
np.random.seed(seed)

3785


In [17]:
# # Experiments
def experiment(Sd_array_total, Cds_total, number_sample):
    Sd_array = []
    Cds = []
    index_samples = random.sample(range(0,140), number_sample)
    index_samples = np.sort(index_samples)
    for i in index_samples:
        Sd_array.append(Sd_array_total[i])
        Cds.append(Cds_total[i])

    Sd_array = np.array(Sd_array)
    Cds = np.array(Cds)

    return Sd_array, Cds, index_samples

sd_array, Cds, index_samples = experiment(Sd_array_total, Cds_total, number_sample)

In [18]:
# # Data visualization of the matrix
from sklearn import neighbors, datasets, manifold
from scipy import linalg
from scipy.linalg import eigh
import matplotlib
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

#### Neighbor search
# we will implement K-nearest neighbor search
def knn_Mat(X, K, t=2.0, dist_metric="euclidean", algorithm="ball_tree"):
    """ compute the neighborhood matrix

    Keyword arguments:
        X: np.array of data, each row(not column) is associated with one car. (A car is represented by a vector)
        K: number of neighbors to seek for each element (the element itself is excluded)
        dist_metric: with which metric to compute the distances
        algorithm: which which algorithm to use

    return:
        the neighborhood matrix
    """

    n, p = X.shape  # n: number of elements, p: dimension (number of data per element)

    knn = neighbors.NearestNeighbors(K + 1, metric=dist_metric, algorithm=algorithm).fit(X)
    distances, nbors = knn.kneighbors(X)  # return k nearest neighbours of each member of X, nbors is array of indices

    return (nbors[:, 1:])  # neighborhood matrix first one is the point itself, which should be excluded


#### calculation of reconstruction weights
def get_weights(X, nbors, reg, K):
    """ compute the weight matrix

    Keyword arguments:
        X: np.array of data, each row is associated with one car
        nbors; neighborhood matrix
        regularized term: avoid the covariance matrix to be singular #regularizer
        K: number of neighbours selected for each car

    return:
        the weight matrix
    """

    n, p = X.shape  # n number of car shapes, n rows and p columns (60 rows and 112000 columns in our case)

    Weights = np.zeros((n, n))

    for i in range(n):

        X_bors = X[nbors[i], :] - X[i]  # Gi in MLLE paper
        cov_nbors = np.dot(X_bors, X_bors.T)

        # regularization terms
        trace = np.trace(cov_nbors)
        if trace > 0:
            R = reg * trace
        else:
            R = reg
        cov_nbors.flat[::K + 1] += R  # [::K+1] -> every K+1 step. Allows adding only on trace
        weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True)  # solution to cov_nbors*weights = 1
        # optimal solution to min(cov_nbors*W)

        # normalizing
        weights = weights / weights.sum()
        # only ponderable weights of neighbours should be different from zero
        #Weights[i, nbors[i]] = weights # put all those weights into a matrix
        Weights[i, nbors[i]] = weights

    return (Weights)


#### Calculate the embedded data using the weights
# calculation of the new embedding
def Y_(Weights, d):
    """ reconstruct data into the low dimensional space

    Keyword arguments:
        Weights: weight matrix
        d: number of dimension in which to project onto (2 in our case)

    return:
        the dataset projected onto d dimensions
    """
    n, p = Weights.shape
    I = np.eye(n)
    m = (I - Weights)
    M = m.T.dot(m)

    eigvals, eigvecs = eigh(M, eigvals=(1, d), overwrite_a=True)
    ind = np.argsort(np.abs(eigvals))

    return (eigvecs[:, ind])  # we only keep the the most important vectors (?? xl)


def LLE_(X, K, d, nbors):
    """ proceed to various LLE steps

    Keyword arguments:
        X: dataset
        K: number of neighbours to seek
        d: on how many dimension to project the dataset

    return:
        the data set projected into d dimensions
    """
    reg = 0.001 # regularized term

    try:
        if nbors == None:
            nbors = knn_Mat(X, K)
    except:
        pass
    Weights = get_weights(X, nbors, reg, K)

    Y = Y_(Weights, d)

    return [Y, Weights, nbors]

X = sd_array
n_sample, sample_dim = np.shape(X)
K = 6 #in our case, K=6 gets the lowest reconstruction error
d = 2
nbors = None
Y, Weights, nbors = LLE_(X, K, d, nbors)
err = np.linalg.norm(Y - np.dot(Weights,Y),'fro')**2 # reconstruction error for Y (low dimensional data)

# print("Reconstruction error: %g" %err)

# extracting the weights of neighbors for all shapes
W_shapes = np.zeros((n_sample,K)) # W_shapes: weights extracted from Weight matrix
for i in range(n_sample):
    for j in range(K):
        n = nbors[i,:][j]
        W_shapes[i,j] = Weights[i,n]
# print(W_shapes)

# Plot result
# fig = plt.figure()
# ax1 = fig.add_subplot(111)
# y0 = Y[:, 0]
# y1 = Y[:, 1]
# ax1.scatter(y0,y1)
# plt.axis('tight')
# plt.xticks([]), plt.yticks([])
# plt.title('Projected data')

C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [19]:
# # Plot result
# fig, ax = plt.subplots()
# #fig = plt.figure()
# # ax1 = fig.add_subplot(111)
# # ax2 = fig.add_subplot(112)
# # ax3 = fig.add_subplot(113)
# # ax4 = fig.add_subplot(114)
# x_b = Y[big, 0]
# y_b = Y[big, 1]
# x_c = Y[classic, 0]
# y_c = Y[classic, 1]
# x_s = Y[small, 0]
# y_s = Y[small, 1]
# x_sp = Y[sports, 0]
# y_sp = Y[sports, 1]
# # ax1.scatter(x_b,y_b)
# # ax2.scatter(x_c,y_c)
# # ax3.scatter(x_s,y_s)
# # ax4.scatter(x_sp,y_sp)
# plt.scatter(x_b,y_b, s=100, marker=">", label='big cars') #triangle
# plt.scatter(x_c,y_c, s=100, marker=(5,0), label='classic cars')#wubianxing
# plt.scatter(x_s,y_s, s=100, marker="+", label='small cars')#plus
# plt.scatter(x_sp,y_sp, s=100, marker=(5,1), label='sports cars')#stars
# plt.legend(loc='lower right')
# plt.axis('tight')
# plt.xticks([]), plt.yticks([])
# plt.title('Projected data')

In [20]:
# # Fast evaluation of the drag coefficient of a new car shape
validation20 = np.genfromtxt('v_rep_20_density145.csv', delimiter=',')
#read models's names
# f = open('validation_20.txt', 'r')
# path_list = []
# for x in f:
#     #print(x)
#     x = x[:-1]
#     path_list.append(x)
#
# # print(path_list)
#
# f.close()

models = []
cd_models = []
for i, f in enumerate(validation20):
        #### Neighbors search for a new added shape
    # we will implement K-nearest neighbor search
    def knn_new(X, K, x_new, t=2.0, dist_metric="euclidean", algorithm="ball_tree"):
        """ compute the neighborhood matrix

        Keyword arguments:
            X: np.array of data, each row(not column) is associated with one car. (A car is represented by a vector)
            K: number of neighbors to seek for each element (the element itself is excluded)
            dist_metric: with which metric to compute the distances
            algorithm: which which algorithm to use

        return:
            the neighborhood matrix
        """

        n, p = X.shape  # n: number of elements, p: dimension (number of data per element)

        knn = neighbors.NearestNeighbors(K + 1, metric=dist_metric, algorithm=algorithm).fit(X)
        nbors = knn.kneighbors([x_new],K + 1, return_distance=False)  # return k nearest neighbours of each member of X, nbors is array of indices

        return (nbors[:, 1:])  # neighborhood matrix first one is the point itself, which should be excluded
    #### calculation of reconstruction weights
    def get_weights_new(x_new, knn_new, K, reg=0.001):
        """ compute the weight matrix

        Keyword arguments:
            x_new: a new vector representing a new car shape
            knn_new: neighborhood of the new car shape
            reg: avoid the covariance matrix to be singular #regularizer
            K: number of neighbours selected for each car

        return:
            the vector of weights for neighboring data points
        """

        X_bors = X[knn_new[0:K], :] - x_new  # Gi in MLLE paper, X[i] = X[i,:] ith row of X
        cov_nbors = np.dot(X_bors, X_bors.T)

        # regularization terms
        trace = np.trace(cov_nbors)
        if trace > 0:
            R = reg * trace
        else:
            R = reg
        cov_nbors.flat[::K + 1] += R  # [::K+1] -> every K+1 step. Allows adding only on trace
        weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True)  # solution to cov_nbors*weights = 1
            # optimal solution to min(cov_nbors*W)

            # normalizing
        weights = weights / weights.sum()

        return (weights)

    start_time_new = time.time()
    # print(f)
    # example of a new car shape
    # knn_new: neighbors of the new shpae in the manifold space
    x_new = f
    knn_new = knn_new(X,K,x_new)[0] #knn_new(X,K,x_new)is a tuple with one element an array
    print(f"This is {knn_new} for {i} model")
    w_new = get_weights_new(x_new, knn_new, K)
    # print(w_new)
    # Calculation of the drag coefficient of the new car
    Cds_nn = np.zeros(K) # drag coefficients of the nearest neighbours
    for j in range(K):
        Cds_nn[j] = Cds[knn_new[j]]
    # print(Cds_nn)
    Cd_new = round(np.dot(w_new, Cds_nn),3)
    # print(knn_new)
    # print(w_new)
    # print(path_list[i], Cd_new)
    # models.append(path_list[i])
    cd_models.append(Cd_new)
    # print(time.time() - start_time_new)


print(index_samples)
# print(models)
cd_models = np.array(cd_models)
# print(cd_models)

C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


This is [ 71  86 111 121 139  20] for 0 model


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


This is [ 86 139  39 126 128  91] for 1 model


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

This is [ 50 136  77  44   4  72] for 17 model


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


This is [ 39 126  86 139  91 128] for 18 model


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


This is [107  67  47   6  40  92] for 19 model
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139]


In [21]:
CFD_Cds = np.genfromtxt('Cds_20.csv', delimiter=',')
# print(CFD_Cds)

relative_error = np.zeros(cd_models.shape)
for i, cd in enumerate(cd_models):
    relative_error[i] = np.abs(cd - CFD_Cds[i])/CFD_Cds[i]

mean_relative_error = np.mean(relative_error)
std_relative_error = np.std(relative_error, ddof=1)

# print('Relative error is {}'.format(relative_error))
print('Experiments for {}'.format(number_sample))
print('Mean relative error is {}'.format(mean_relative_error))
print('STD relative error is {}'.format(std_relative_error))
print('===================================================')

f = open('Experiments_all.txt', 'a')
f.writelines('random seed {} \n'.format(seed))
f.writelines('number of models {} \n'.format(number_sample))
f.writelines('index {} \n'.format(index_samples))
f.writelines('Mean relative error is {} \n'.format(mean_relative_error))
f.writelines('STD relative error is {} \n'.format(std_relative_error))
f.writelines('=================================================== \n')
f.close()

f1 = open('Experiments_50_60-140_2.txt', 'a')
f1.writelines('50 density number of models {}'.format(number_sample) + ' Mean relative error is, {}'.format(mean_relative_error) + ', STD relative error is, {}, \n'.format(std_relative_error))
f1.close()


Experiments for 140
Mean relative error is 0.05465884953422965
STD relative error is 0.04251272123725548
